<a href="https://colab.research.google.com/github/clam004/notebook_tutorials/blob/main/intro_autoregressive_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install transformers datasets

In [ ]:
#sys libs
import os
import sys
import random
import time
import json

#data manupulation libs
import numpy as np

#string manupulation libs
import re
import string

#torch libs
import torch
print('torch.cuda.device_count()', torch.cuda.device_count())
print('torch.cuda.empty_cache()', torch.cuda.empty_cache())

#huggingface transformers
import transformers
from transformers import set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel

print(transformers.__version__)

# seeds
set_seed(42)
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

%load_ext autoreload
%autoreload 2
%matplotlib inline

torch.cuda.device_count() 1
torch.cuda.empty_cache() None
4.20.1


In [ ]:
prompt_list = [
    "this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.",
    "[human]: hello",
    "[robot]: how are you today",
    "[human]: im well, but how are you?",
    "[robot]:",
]

prompt = '\n'.join(prompt_list)
print(prompt)

this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.
[human]: hello
[robot]: how are you today
[human]: im well, but how are you?
[robot]:


In [ ]:
pretrained_hf_model = 'gpt2-large' # 'gpt2-xl' # 

tokenizer = GPT2Tokenizer.from_pretrained(
    pretrained_hf_model,
    pad_token='<|endoftext|>',
)

cache_dir = os.path.join(
    "./modelstates/hugface_models/",
    pretrained_hf_model,
)

print("cache_dir=", cache_dir)

model = GPT2LMHeadModel.from_pretrained(
    pretrained_hf_model,
    cache_dir=cache_dir,
)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("num_params", num_params/1e6, "Million Parameters")

cache_dir= ./modelstates/hugface_models/gpt2-large
num_params 774.03008 Million Parameters


In [ ]:
prompt_tokens = tokenizer(
    prompt,
    return_tensors="pt",
)

# you must have a GPU to use this 2 lines
prompt_ids = prompt_tokens.input_ids.cuda()
prompt_attention_masks = prompt_tokens.attention_mask.cuda()
model = model.cuda()

prompt_len = prompt_ids.shape[1]

output_ids = model.generate(
    prompt_ids,
    attention_mask = prompt_attention_masks,
    max_length=prompt_len+32,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = tokenizer.batch_decode(output_ids)[0]
print(' ')
print(generated_text)

 
this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.
[human]: hello
[robot]: how are you today
[human]: im well, but how are you?
[robot]: i am very well, but i am very tired
[human]: i am very tired too
[robot]: i am very tired too
[human


In [ ]:
reply = generated_text[len(prompt):]
reply = reply[:reply.index('\n[human]')]
reply

' i am very well, but i am very tired'

In [ ]:
prompt_list_ = prompt_list
reply_ = reply

stop_seq = '[human]:'
for i in range(5):

  human_utterance = input('[human]: ')
  
  prompt_list_ = prompt_list_[:-1] + ['[robot]:'+reply_] +['[human]: '+human_utterance]
  prompt_ = '\n'.join(prompt_list_)
  #print(prompt_)
 
  prompt_tokens_ = tokenizer(
        prompt_,
        return_tensors="pt",
    )

  # you must have a GPU to use this 2 lines
  prompt_ids_ = prompt_tokens_.input_ids.cuda()
  prompt_attention_masks = prompt_tokens_.attention_mask.cuda()
  model = model.cuda()

  prompt_len = prompt_ids_.shape[1]

  output_ids = model.generate(
      prompt_ids_,
      attention_mask = prompt_attention_masks,
      max_length=prompt_len+32,
      pad_token_id=tokenizer.eos_token_id,
  )

  generated_text_ = tokenizer.batch_decode(output_ids)[0]
  reply_ = generated_text_[len(prompt_):]
  #print(reply_)

  if stop_seq in reply_:
    reply_ = reply_[:reply_.index(stop_seq)]

  print(reply_)


[human]: whats your name?

[robot]: my name is [human]

[human]: how can that be your name
?

[robot]: i am a human

[robot]: i am a human child




KeyboardInterrupt: ignored